In [15]:
# import ml tranformation libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [16]:
#load data
df = pd.read_csv('data/data.csv', sep=';')  # Add semicolon separator

In [17]:
# transform data using pipelines and column tranformers. 
# First we find the numerical and categorical columns, to do that we need to expect the number of unique values for each column, we do this since
# we dont necesarily know if the categorical columns are of object or int type, so we may need to set a threshold for unique values to consider a column categorical.

print("Number of unique values per column:")
print(df.nunique())
print(f"\nTotal columns: {len(df.columns)}")

print("\n" + "="*60)
print("Data types per column:")
print(df.dtypes)

print("\n" + "="*60)
print("Summary - Column Name | Data Type | Unique Values:")
print("="*60)
for col in df.columns:
    print(f"{col:50} | {str(df[col].dtype):10} | {df[col].nunique()}")


Number of unique values per column:
Marital status                                      6
Application mode                                   18
Application order                                   8
Course                                             17
Daytime/evening attendance\t                        2
Previous qualification                             17
Previous qualification (grade)                    101
Nacionality                                        21
Mother's qualification                             29
Father's qualification                             34
Mother's occupation                                32
Father's occupation                                46
Admission grade                                   620
Displaced                                           2
Educational special needs                           2
Debtor                                              2
Tuition fees up to date                             2
Gender                                        

In [18]:
# Separate numerical and categorical columns
# Strategy: Use threshold for unique values + manual verification

# Set threshold - if unique values <= threshold, treat as categorical
THRESHOLD = 15

# First pass: automatic separation based on threshold
potential_categorical = []
potential_numerical = []

for col in df.columns:
    if col == 'Target':  # Skip target variable
        continue
    
    unique_count = df[col].nunique()
    if unique_count <= THRESHOLD:
        potential_categorical.append(col)
    else:
        potential_numerical.append(col)

print(f"Threshold set to: {THRESHOLD} unique values")
print("\n" + "="*70)
print("POTENTIAL CATEGORICAL COLUMNS (unique values <= threshold):")
print("="*70)
for col in potential_categorical:
    print(f"{col:50} | Unique: {df[col].nunique():3} | Type: {df[col].dtype}")

print("\n" + "="*70)
print("POTENTIAL NUMERICAL COLUMNS (unique values > threshold):")
print("="*70)
for col in potential_numerical:
    print(f"{col:50} | Unique: {df[col].nunique():4} | Type: {df[col].dtype}")

print("\n" + "="*70)
print("IMPORTANT: Review the lists above and manually adjust if needed!")
print("Some columns might need special treatment based on domain knowledge.")
print("="*70)


Threshold set to: 15 unique values

POTENTIAL CATEGORICAL COLUMNS (unique values <= threshold):
Marital status                                     | Unique:   6 | Type: int64
Application order                                  | Unique:   8 | Type: int64
Daytime/evening attendance	                        | Unique:   2 | Type: int64
Displaced                                          | Unique:   2 | Type: int64
Educational special needs                          | Unique:   2 | Type: int64
Debtor                                             | Unique:   2 | Type: int64
Tuition fees up to date                            | Unique:   2 | Type: int64
Gender                                             | Unique:   2 | Type: int64
Scholarship holder                                 | Unique:   2 | Type: int64
International                                      | Unique:   2 | Type: int64
Curricular units 1st sem (without evaluations)     | Unique:  11 | Type: int64
Curricular units 2nd sem (without e

In [19]:
# Manual verification and final column lists
# Review the output above and adjust these lists as needed

# Categorical columns (to be One-Hot Encoded)
cat_vals = [
    'Marital status',
    'Application mode',
    'Application order',
    'Course',
    'Daytime/evening attendance\t',
    'Previous qualification',
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder',
    'International'
]

# Numerical columns (to be Standard Scaled)
num_vals = [
    'Previous qualification (grade)',
    'Admission grade',
    'Age at enrollment',
    'Curricular units 1st sem (credited)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)',
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP'
]

print(f"Categorical columns: {len(cat_vals)}")
print(f"Numerical columns: {len(num_vals)}")
print(f"Total: {len(cat_vals) + len(num_vals)}")
print(f"Expected (excluding Target): {len(df.columns) - 1}")

# Verify all columns are accounted for
all_cols = set(df.columns) - {'Target'}
defined_cols = set(cat_vals + num_vals)
missing = all_cols - defined_cols
extra = defined_cols - all_cols

if missing:
    print(f"\n⚠️  WARNING: Missing columns: {missing}")
if extra:
    print(f"\n⚠️  WARNING: Extra columns (not in df): {extra}")
if not missing and not extra:
    print("\n✓ All columns accounted for!")


Categorical columns: 18
Numerical columns: 18
Total: 36
Expected (excluding Target): 36

✓ All columns accounted for!


In [20]:
# Check for missing values first
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nTotal missing values: {df.isnull().sum().sum()}")


Missing values per column:
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance\t                      0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship holder                   

In [21]:
# Create preprocessing pipelines with imputation

# Numerical pipeline: Impute with mean, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scale', StandardScaler())
])

# Categorical pipeline: Impute with most frequent, then one-hot encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine both pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_vals),
    ('cat', cat_pipeline, cat_vals)
])

print("Preprocessing pipelines created!")
print("\nNumerical pipeline: Impute with MEAN → StandardScaler")
print("Categorical pipeline: Impute with MOST_FREQUENT → OneHotEncoder")
print(f"\nNumerical features: {len(num_vals)}")
print(f"Categorical features: {len(cat_vals)}")


Preprocessing pipelines created!

Numerical pipeline: Impute with MEAN → StandardScaler
Categorical pipeline: Impute with MOST_FREQUENT → OneHotEncoder

Numerical features: 18
Categorical features: 18


In [22]:
# Split data into features (X) and target (y) BEFORE transformation
X = df.drop('Target', axis=1)
y = df['Target']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Maintain class distribution in splits
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"\nFeatures: {X_train.shape[1]}")
print(f"Target distribution in training set:")
print(y_train.value_counts())
print(f"\nTarget distribution in test set:")
print(y_test.value_counts())


Training set size: 3539 samples
Test set size: 885 samples

Features: 36
Target distribution in training set:
Target
Graduate    1767
Dropout     1137
Enrolled     635
Name: count, dtype: int64

Target distribution in test set:
Target
Graduate    442
Dropout     284
Enrolled    159
Name: count, dtype: int64


In [23]:
# Fit the preprocessor on training data and transform both train and test
# IMPORTANT: Fit only on training data to avoid data leakage!

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)  # Use fitted preprocessor, don't fit again

print("Data transformation complete!")
print(f"\nOriginal training data shape: {X_train.shape}")
print(f"Transformed training data shape: {X_train_transformed.shape}")
print(f"\nOriginal test data shape: {X_test.shape}")
print(f"Transformed test data shape: {X_test_transformed.shape}")
print(f"\nNote: Column count increased due to One-Hot Encoding of categorical variables")


Data transformation complete!

Original training data shape: (3539, 36)
Transformed training data shape: (3539, 251)

Original test data shape: (885, 36)
Transformed test data shape: (885, 251)

Note: Column count increased due to One-Hot Encoding of categorical variables


In [ ]:
# Save preprocessor and data splits for reproducibility
import joblib

# Save the fitted preprocessor (can reuse for new data later)
joblib.dump(preprocessor, 'data/preprocessor.joblib')

# Save raw splits (before transformation)
X_train.to_csv('data/X_train_raw.csv', index=False)
X_test.to_csv('data/X_test_raw.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

print("Data saved successfully!")
print(f"  - data/preprocessor.joblib (fitted transformer)")
print(f"  - data/X_train_raw.csv: {X_train.shape}")
print(f"  - data/X_test_raw.csv: {X_test.shape}")
print(f"  - data/y_train.csv: {len(y_train)} rows")
print(f"  - data/y_test.csv: {len(y_test)} rows")